In [1]:
import sunpy 
import sunpy.map
import astropy
import numpy as np
import bottleneck as bn
from glob import glob

In [2]:
date = '20221026'

In [3]:
eui_files = sorted(glob(f"../../src/EUI/HRI/euv174/{date}/coalign_step/*.fits"))
eui_map_seq_coalign = sunpy.map.Map(eui_files[:],sequence=True)

In [4]:
len_segments = 120
n_segments = np.ceil(len(eui_map_seq_coalign)/len_segments).astype(int)
len_window = 6

In [5]:
eui_map_seq_segs = []
for ii in range(n_segments):
    if ii == 0:
        eui_map_seq_segs.append(eui_map_seq_coalign[0:len_segments])
    elif ii == n_segments - 1:
        eui_map_seq_segs.append(eui_map_seq_coalign[ii*len_segments - 2*len_window:])
    else:
        eui_map_seq_segs.append(eui_map_seq_coalign[ii*len_segments - 2*len_window:(ii+1)*len_segments])


In [6]:
for ii, eui_map_seq_seg_ in enumerate(eui_map_seq_segs):
    eui_map_array = eui_map_seq_seg_.as_array()
    eui_map_array_boxcar_mean = bn.move_mean(eui_map_array,window=len_window,axis=2,min_count=1)

    if ii == 0:
        for jj in range(len_segments - len_window):
            new_map = sunpy.map.Map(eui_map_array_boxcar_mean[:,:,jj],eui_map_seq_seg_.maps[jj].meta)
            new_map.save(f"../../src/EUI/HRI/euv174/{date}/coalign_step_boxcar/eui_map_seq_coalign_boxcar_{jj:03}.fits",overwrite=True)
    
    elif ii == n_segments - 1:
        for jj in range(len_window, len(eui_map_seq_seg_)):
            new_map = sunpy.map.Map(eui_map_array_boxcar_mean[:,:,jj],eui_map_seq_seg_.maps[jj].meta)
            new_map.save(f"../../src/EUI/HRI/euv174/{date}/coalign_step_boxcar/eui_map_seq_coalign_boxcar_{jj+ii*len_segments - 2*len_window:03}.fits",overwrite=True)
    
    else:
        for jj in range(len_window, len(eui_map_seq_seg_) - len_window):
            new_map = sunpy.map.Map(eui_map_array_boxcar_mean[:,:,jj],eui_map_seq_seg_.maps[jj].meta)
            new_map.save(f"../../src/EUI/HRI/euv174/{date}/coalign_step_boxcar/eui_map_seq_coalign_boxcar_{jj+ii*len_segments - 2*len_window:03}.fits",overwrite=True)
